In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [3]:
# 데이터 출처 - http://data.nsdi.go.kr/dataset/20180927ds0059 (국가공간정보포털-행정경계(읍면동))
# shp 파일 경로
shp_file_path = "철새/shape_export_20240111_152226.shp"

# shp 파일을 GeoDataFrame으로 읽기
gdf = gpd.read_file(shp_file_path)
gdf

,taxon_f_nm,col_id,ktsn,group_nm,group,taxon_nm,geometry
0,Acrocephalus orientalis,NIBRGR0000134240,120000002037,Á¶·ù,AV,°³°³ºñ,POINT (13962327.839 4118077.974)
1,Acrocephalus orientalis,NIBRGR0000134208,120000002037,Á¶·ù,AV,°³°³ºñ,POINT (14131607.369 4517580.854)
2,"Streptopelia orietalis (Latham, 1790)",NIBRAV0000102142,120000001825,Á¶·ù,AV,¸äºñµÑ±â,POINT (14076460.621 4504753.434)
3,Egretta alba,NIBRAV0000101544,120000080653,Á¶·ù,AV,Áß´ë¹é·Î,POINT (14076460.621 4504753.434)
4,"Emberiza spodocephala Pallas, 1776",NIBRGR0000134227,120000002230,Á¶·ù,AV,ÃË»õ,POINT (13962327.839 4118077.974)
...,...,...,...,...,...,...,...
646,"Paradoxornis webbianus (Gould, 1852)",NIBRGR0000134497,120000002061,Á¶·ù,AV,ºÓÀº¸Ó¸®¿À¸ñ´«ÀÌ,POINT (14042015.897 4394934.432)
647,"Paradoxornis webbianus (Gould, 1852)",NIBRGR0000134498,120000002061,Á¶·ù,AV,ºÓÀº¸Ó¸®¿À¸ñ´«ÀÌ,POINT (14042015.897 4394934.432)
648,"Paradoxornis webbianus (Gould, 1852)",NIBRGR0000134499,120000002061,Á¶·ù,AV,ºÓÀº¸Ó¸®¿À¸ñ´«ÀÌ,POINT (14042015.897 4394934.432)
649,"Emberiza tristrami Swinhoe, 1870",NIBRGR0000134500,120000002219,Á¶·ù,AV,Èò¹è¸ä»õ,POINT (14042015.897 4394934.432)
